# Setup for Object Detection API and tensorflow

##Check Tensorflow version

In [ ]:
from google.colab import drive
import tensorflow as tf
import os

print(tf.version.VERSION)
drive.mount('/content/drive/')
# os.environ['WS'] = '/content/drive/MyDrive/colab_object_detection_setup'
os.chdir('/content/drive/MyDrive/colab_object_detection_setup')

## Install Object Detection API

### Clone Repository

In [ ]:
!git clone https://github.com/tensorflow/models.git Tensorflow/models

### Build Protokollbuffers

In [ ]:
%cd Tensorflow/models/research
!protoc object_detection/protos/*.proto --python_out=.

### Install Dependencies

In [ ]:
# !cp object_detection/packages/tf2/setup.py .
!python -m pip install .

### Verify installation

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# Create workspace for training

In [ ]:
os.chdir('/content/drive/MyDrive/colab_object_detection_setup/Tensorflow')
!mkdir workspace
!mkdir workspace/demo_training
!mkdir workspace/demo_training/compressed_records
!mkdir workspace/demo_training/annotations
!mkdir workspace/demo_training/exported-models
!mkdir workspace/demo_training/exported-models/custom_SSD_MobileNet_V2_FPNLite_320x320
!mkdir workspace/demo_training/images/
!mkdir workspace/demo_training/images/test
!mkdir workspace/demo_training/images/train
!mkdir workspace/demo_training/models
!mkdir workspace/demo_training/models/custom_SSD_MobileNet_V2_FPNLite_320x320
!mkdir workspace/demo_training/pre-trained-models
!touch workspace/demo_training/README.md

## Load exported records from roboflow

Before executing the code below upload the zip file to the compressed_records directory

In [ ]:
%cd workspace/demo_training/
!unzip compressed_records/microwunderland.v1i.tfrecord.zip -d compressed_records/
!mv compressed_records/train compressed_records/test annotations/

## Download model from [tesnorflow model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

Selected model for this workspace is ***SSD MobileNet V2 FPNLite 320x320***

In [ ]:
!wget -P pre-trained-models/ http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xf pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz -C pre-trained-models/

## Configure the Training Pipeline

In [ ]:
!cp pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config models/custom_SSD_MobileNet_V2_FPNLite_320x320

Copy original pipline config to the directory where the cutomized model will be saved. Then configure the file so it fits your training data. Attributes that should be checked are listed below:

+ num_classes
+ batch_size
+ fine_tune_checkpoint
+ fine_tune_checkpoint_type
+ use_bfloat16
+ label_map_path
+ input_path
+ metrics_set
+ use_moving_averages
+ label_map_path
+ input_path

## Train the Model

copy training script

In [ ]:
# !cp /content/drive/MyDrive/colab_object_detection_setup/Tensorflow/models/research/object_detection/model_main_tf2.py .
!cp /content/drive/MyDrive/colab_object_detection_setup/Tensorflow/models/research/object_detection/exporter_main_v2.py .

start tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Add to tf.keras callback
tensorboard_callback = tf.keras.callbacks.TensorBoard("models/custom_SSD_MobileNet_V2_FPNLite_320x320", update_freq='batch')

In [ ]:
%tensorboard --logdir models/custom_SSD_MobileNet_V2_FPNLite_320x320

## Start training

In [ ]:
# Training script
!python model_main_tf2.py --model_dir=models/custom_SSD_MobileNet_V2_FPNLite_320x320 --pipeline_config_path=models/custom_SSD_MobileNet_V2_FPNLite_320x320/pipeline.config

## Export

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/custom_SSD_MobileNet_V2_FPNLite_320x320/pipeline.config --trained_checkpoint_dir models/custom_SSD_MobileNet_V2_FPNLite_320x320/ --output_directory exported-models/custom_SSD_MobileNet_V2_FPNLite_320x320

## Convert to TF-Light Format

Export tflight graph

In [ ]:
%cd /content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training
# Export script
!python export_tflite_graph_tf2.py \
    --pipeline_config_path models/custom_SSD_MobileNet_V2_FPNLite_320x320/pipeline.config \
    --trained_checkpoint_dir models/custom_SSD_MobileNet_V2_FPNLite_320x320/ \
    --output_directory exported-models/TF-Light

In [ ]:
model_path = "/content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training/exported-models/custom_SSD_MobileNet_V2_FPNLite_320x320_v2/saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()

In [ ]:
# !mkdir /content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training/exported-models/TF-Light/
tflite_model_path = "/content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training/exported-models/TF-Light/tflite.tf"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)


## Test base model inference

In [ ]:
# load model
model_path = "/content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training/exported-models/custom_SSD_MobileNet_V2_FPNLite_320x320/saved_model"
model = tf.saved_model.load(model_path)


In [ ]:
import cv2


# load image
path = "/content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training/images/valid/55_jpg.rf.ab54b108574648dadf0bc9abf8317f80.jpg"
image = cv2.imread(path)



In [ ]:
# do inference

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]
prediction = model(input_tensor)

print(prediction)

In [ ]:


def draw_bounds(image,index):
  start_point = (int(320*prediction["detection_boxes"][0][index][0]),int(320*prediction["detection_boxes"][0][index][1]))
  end_point = (int(320*prediction["detection_boxes"][0][index][2]),int(320*prediction["detection_boxes"][0][index][3]))
  thickness = 2
  color = (255,0,0)
  print(f"start: {start_point}")
  print(f"end: {end_point}")
  return cv2.rectangle(image, start_point, end_point, color, thickness)

In [ ]:
from google.colab.patches import cv2_imshow

last_reliable_prediction = 0
confidence_limit = 0.7
image = cv2.imread(path)
image = cv2.rotate(image,rotateCode = cv2.ROTATE_90_CLOCKWISE)
image = cv2.flip(image,1)

for last_reliable_prediction,score in enumerate(prediction["detection_scores"][0]):
  if score < confidence_limit:
    break

for i in range(0,last_reliable_prediction):
  print(prediction["detection_scores"][0][i])
  print(prediction["detection_classes"][0][i])
  print(prediction["detection_boxes"][0][i])
  print("--------------------------------------\n")
  image = draw_bounds(image,i)


# Displaying the image
cv2_imshow(image)

## Test TF-Light Model inference

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from google.colab.patches import cv2_imshow

Load TF-Light Model

In [ ]:
tflite_model_path = "/content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training/exported-models/TF-Light/tflite.tf"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

get input and output tensors

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

load image

In [ ]:
# load image
mean = 127.5
std = 127.5
path = "/content/drive/MyDrive/colab_object_detection_setup/Tensorflow/workspace/demo_training/images/valid/96_jpg.rf.15494bf46bd50f4690022f66dfef10c0.jpg"
image = cv2.imread(path)
input_data = np.expand_dims(image,axis=0)
normalized_image_array = (np.float32(input_data) - mean)/std
print(input_details[0]['dtype'])

predict

In [ ]:
interpreter.set_tensor(input_details[0]['index'], normalized_image_array)
interpreter.invoke()

# Abrufen der Vorhersageergebnisse
boxes = (interpreter.get_tensor(output_details[1]['index'])[0])
classes = (interpreter.get_tensor(output_details[3]['index'])[0])
confidence = (interpreter.get_tensor(output_details[0]['index'])[0])

results = [boxes,classes,confidence]

In [ ]:
image = cv2.imread(path)
threshold = 0.7
color = (0,255,0)
line_thickness = 1
image_shape = image.shape
image = cv2.rotate(image,rotateCode = cv2.ROTATE_90_CLOCKWISE)
image = cv2.flip(image,1)

for i in range(0,len(results[1])):
  if(results[2][i] < threshold):
    break

  pos1 = ( int(image_shape[0]*results[0][i][0]), int(image_shape[1]*results[0][i][1]))
  pos2 = ( int(image_shape[0]*results[0][i][2]), int(image_shape[1]*results[0][i][3]))
  cv2.rectangle(image, pos1, pos2, color, line_thickness)

print(i)
cv2_imshow(image)